In [1]:
from qtpg.team import Team
from qtpg.learner import Learner
from qtpg.program import Program
from qtpg.rule import Rule
from qtpg.figure13 import Figure13
from qtpg.search_manager import SearchManager
import numpy as np
import uuid
import copy

In [2]:
# env params
memorySize = 20
legalMove = 0.1
illegalMove = -0.01
outOfBounds = -0.01
memoryRepeat = 0#-0.01
goalReached = 1000

env = Figure13(5, 5, (2, 4), (0, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)

In [10]:
numGens = 50
maxTeamPool = 5
# init search manager and starting teams
searchManager = SearchManager(maxTeamPool)

# init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
# init_rule = Rule(uuid.uuid4(), [0, 0, 0, 0], 0, 0)
# init_learner = Learner(uuid.uuid4(), init_rule)
# (init_team.learners).append(init_learner)
# init_team.mostRecent = init_learner
# (searchManager.teamPool).append(init_team)

init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
init_rule = Rule(uuid.uuid4(), [0, 0, 0, 0], 0, 0)
init_learner = Learner(uuid.uuid4(), init_rule)
(init_team.learners).append(init_learner)
init_team.mostRecent = init_learner
(searchManager.teamPool).append(init_team)

# print('-----init pop start-----')
# for team in searchManager.teamPool:
#     print(f'Team id: {team.id}')
#     print('Learners:')
#     for learner in team.learners:
#         print(learner.id)
# print('-----init pop end-----')

for gen in range(numGens):
    toEvaluateAfterGen = []
    print(f'>>>>>>>>>>>>>>>>>>>>>>>Gen {gen+1} start!>>>>>>>>>>>>>>>>>>>>>>>')
#     print('Teams competing in this round:')
#     for team in searchManager.teamPool:
#         print(f'Team id: {team.id}')
    for parent in searchManager.teamPool:
        # only run if the team has not yet won
        if parent not in searchManager.winners:
            print(f'Parent: {parent.id}')
            env.current_state = parent.start_state
            
            # make offspring from parent, then let that offspring make a new rule
            child = copy.deepcopy(parent)
            child.id = uuid.uuid4()
            win = child.search(env)
            print(f'Child: {child.id}')
            print('Start Child Regions --------')
            for learner in child.learners:
                print(learner.program.rule.region)
            print('End Child Regions --------')
            print(f'Child Fitness: {child.fitness}')
#             print(win)   
            toEvaluateAfterGen.append([child, win])
        else:
            print(f'Skipping Team: {parent.id}')
    
    for data in toEvaluateAfterGen:
        searchManager.evaluate_team(data)
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print(f'End of Gen {gen+1} Regions')
    for i in range(len(searchManager.teamPool)):
        print(f'Team: {i+1} --> {searchManager.teamPool[i].fitness}')
        for learner in searchManager.teamPool[i].learners:
            print(learner.program.rule.region)
    print(f'>>>>>>>>>>>>>>>>>>>>>>>Gen {gen+1} end!>>>>>>>>>>>>>>>>>>>>>>>')
for team in searchManager.winners:
    for learner in team.learners:
        print(learner.program.rule.region)

>>>>>>>>>>>>>>>>>>>>>>>Gen 1 start!>>>>>>>>>>>>>>>>>>>>>>>
Parent: 60f1e5b1-d4ec-4838-ad53-e64854fb730a
Child: baf18813-517a-45c1-b7d4-95b6592351a3
Start Child Regions --------
[0, 0, 0, 0]
[0, 0, 1, 3]
End Child Regions --------
Child Fitness: 0.30000000000000004
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
End of Gen 1 Regions
Team: 1 --> 0.30000000000000004
[0, 0, 0, 0]
[0, 0, 1, 3]
Team: 2 --> 0.30000000000000004
[0, 0, 0, 0]
[0, 0, 1, 3]
Team: 3 --> 0.30000000000000004
[0, 0, 0, 0]
[0, 0, 1, 3]
Team: 4 --> 0.30000000000000004
[0, 0, 0, 0]
[0, 0, 1, 3]
Team: 5 --> 0.30000000000000004
[0, 0, 0, 0]
[0, 0, 1, 3]
>>>>>>>>>>>>>>>>>>>>>>>Gen 1 end!>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>>>>>>>>>Gen 2 start!>>>>>>>>>>>>>>>>>>>>>>>
Parent: baf18813-517a-45c1-b7d4-95b6592351a3
Child: 9b524e5a-9645-44e8-a6f8-e68407d086e0
Start Child Regions --------
[0, 0, 0, 0]
[0, 0, 1, 1]
[1, 2, 0, 2]
End Child Regions --------
Child Fitness: 0.5900000000000001
Parent: baf18813-517a-45c

Parent: baf18813-517a-45c1-b7d4-95b6592351a3
Child: 2647b72b-b990-40a1-a089-e35424306f25
Start Child Regions --------
[0, 0, 0, 0]
[0, 0, 1, 1]
[1, 2, 0, 2]
End Child Regions --------
Child Fitness: 0.5900000000000001
Parent: baf18813-517a-45c1-b7d4-95b6592351a3
Child: 11bf299f-73a9-4967-940c-002f7590f334
Start Child Regions --------
[0, 0, 0, 0]
[0, 0, 5, 3]
[1, 4, 0, 0]
End Child Regions --------
Child Fitness: 0.27
Parent: baf18813-517a-45c1-b7d4-95b6592351a3
Child: a9f01856-2345-44d9-ba07-978c1328e0aa
Start Child Regions --------
[0, 0, 0, 0]
[0, 0, 1, 1]
[1, 2, 0, 2]
End Child Regions --------
Child Fitness: 0.5900000000000001
Parent: baf18813-517a-45c1-b7d4-95b6592351a3
Child: 42298e25-517f-4c7d-8c51-5d0f3a3223c4
Start Child Regions --------
[0, 0, 0, 0]
[0, 0, 5, 3]
[1, 4, 0, 0]
End Child Regions --------
Child Fitness: 0.27
Parent: baf18813-517a-45c1-b7d4-95b6592351a3
Child: af35876c-b261-468c-8238-587e9b90df80
Start Child Regions --------
[0, 0, 0, 0]
[0, 0, 1, 1]
[1, 2, 0, 2]

Child: afcd8bef-a5f7-4fa5-affc-9ef73c9360fe
Start Child Regions --------
[0, 0, 0, 0]
[0, 0, 5, 3]
[1, 4, 0, 0]
End Child Regions --------
Child Fitness: 0.27
Parent: baf18813-517a-45c1-b7d4-95b6592351a3
Child: fd570594-b9f7-4929-bc8c-2948f956875f
Start Child Regions --------
[0, 0, 0, 0]
[0, 0, 5, 3]
[1, 4, 0, 0]
End Child Regions --------
Child Fitness: 0.27
Parent: baf18813-517a-45c1-b7d4-95b6592351a3
Child: 5e69ed7d-e396-4057-bf91-c32aebe1ff2a
Start Child Regions --------
[0, 0, 0, 0]
[0, 0, 1, 1]
[1, 2, 0, 2]
End Child Regions --------
Child Fitness: 0.5900000000000001
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
End of Gen 48 Regions
Team: 1 --> 0.30000000000000004
[0, 0, 0, 0]
[0, 0, 1, 3]
Team: 2 --> 0.30000000000000004
[0, 0, 0, 0]
[0, 0, 1, 3]
Team: 3 --> 0.30000000000000004
[0, 0, 0, 0]
[0, 0, 1, 3]
Team: 4 --> 0.30000000000000004
[0, 0, 0, 0]
[0, 0, 1, 3]
Team: 5 --> 0.30000000000000004
[0, 0, 0, 0]
[0, 0, 1, 3]
>>>>>>>>>>>>>>>>>>>>>>>Gen 48 end!>>>>>>>>